In [23]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px
import hvplot.pandas
import panel as pn
import os
from pathlib import Path
# connectionString = "dbname='project01' user='pgadmin@fintech-postgres' host='fintech-postgres.postgres.database.azure.com' password='CgYRz!)[4pfB' port='5432' sslmode='true'"
engine = create_engine(
    "postgresql://pgadmin@fintech-postgres:CgYRz!)[4pfB@fintech-postgres.postgres.database.azure.com:5432/project01")
query = """
select portfolio_name, weight, "date" as close_date,
    close_price, ticker_name, covidineffect
from portfolio p
    inner join portfolio_tickers pt
    on p.portfolio_id = pt.portfolio_id
    inner join price
    on pt.ticker_id = price.ticker_id
;"""
df = pd.read_sql(query, engine)
df.head(250)

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect


In [8]:
df["porttick"] = df["portfolio_name"] + "-" + df["ticker_name"]
df.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
0,food,0.25,2019-02-19,41.85,ADM,False,food-ADM
1,food,0.25,2019-02-20,41.97,ADM,False,food-ADM
2,food,0.25,2019-02-21,42.57,ADM,False,food-ADM
3,food,0.25,2019-02-22,42.49,ADM,False,food-ADM
4,food,0.25,2019-02-25,42.43,ADM,False,food-ADM


In [9]:
dfCovidNo = df.loc[df["covidineffect"] == False]
dfCovidNoPivot = dfCovidNo.pivot(index="close_date", columns="porttick", values="close_price")
dfCovidNoPivot.head()


porttick,S&P500-SP500,communication-CHTR,communication-CMCSA,communication-T,communication-VZ,financial-BAC,financial-GS,financial-PYPL,financial-V,food-ADM,...,food-HRL,food-TSN,hedge-BTCUSD,hedge-ETHUSD,hedge-GLD,hedge-SLV,pharma-ABT,pharma-JNJ,pharma-MRK,pharma-PFE
close_date,,,,,,,,,,,,,,,,,,,,,
2019-02-19,2779.76,348.43,37.55,30.63,55.44,29.13,198.67,95.02,144.44,41.85,...,43.30,62.94,3888.01,142.69,126.70,15.02,75.00,135.69,79.24,42.60
2019-02-20,2784.70,349.76,37.79,30.85,55.76,29.41,198.60,94.73,144.73,41.97,...,43.13,63.60,3938.99,147.19,126.48,15.07,75.54,136.35,79.43,42.20
2019-02-21,2774.88,349.54,38.47,30.83,56.15,29.29,196.36,94.96,144.00,42.57,...,41.99,63.06,3897.71,144.21,125.05,14.83,75.14,135.42,79.83,42.14
2019-02-22,2792.67,350.08,38.61,31.15,56.92,29.08,196.00,95.98,145.87,42.49,...,42.91,63.32,3942.02,146.68,125.50,14.95,76.13,136.60,80.77,42.96
2019-02-25,2796.11,348.90,38.57,31.13,56.78,29.27,198.65,96.70,146.06,42.43,...,42.53,62.32,3818.79,137.64,125.37,14.90,76.86,136.20,80.38,43.08


In [4]:
dfCovid = df.loc[df["covidineffect"] == True]
dfCovidPivot = dfCovid.pivot(index="close_date", columns='ticker_name', values="close_price")
dfCovidPivot.head()

ticker_name,ABT,ADM,BAC,BTCUSD,CAG,CHTR,CMCSA,ETHUSD,GLD,GS,...,JNJ,MRK,PFE,PYPL,SLV,SP500,T,TSN,V,VZ
close_date,,,,,,,,,,,,,,,,,,,,,
2019-02-19,75.00,41.85,29.13,3888.01,24.01,348.43,37.55,142.69,126.70,198.67,...,135.69,79.24,42.60,95.02,15.02,2779.76,30.63,62.94,144.44,55.44
2019-02-20,75.54,41.97,29.41,3938.99,24.03,349.76,37.79,147.19,126.48,198.60,...,136.35,79.43,42.20,94.73,15.07,2784.70,30.85,63.60,144.73,55.76
2019-02-21,75.14,42.57,29.29,3897.71,24.07,349.54,38.47,144.21,125.05,196.36,...,135.42,79.83,42.14,94.96,14.83,2774.88,30.83,63.06,144.00,56.15
2019-02-22,76.13,42.49,29.08,3942.02,23.38,350.08,38.61,146.68,125.50,196.00,...,136.60,80.77,42.96,95.98,14.95,2792.67,31.15,63.32,145.87,56.92
2019-02-25,76.86,42.43,29.27,3818.79,23.31,348.90,38.57,137.64,125.37,198.65,...,136.20,80.38,43.08,96.70,14.90,2796.11,31.13,62.32,146.06,56.78


In [10]:
dfJohn = df.loc[df["portfolio_name"] == "pharma"]
dfJohn.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
8876,pharma,0.25,2019-02-19,75.00,ABT,False,pharma-ABT
8877,pharma,0.25,2019-02-20,75.54,ABT,False,pharma-ABT
8878,pharma,0.25,2019-02-21,75.14,ABT,False,pharma-ABT
8879,pharma,0.25,2019-02-22,76.13,ABT,False,pharma-ABT
8880,pharma,0.25,2019-02-25,76.86,ABT,False,pharma-ABT


In [11]:
dfSusan = df.loc[df["portfolio_name"] == "food"]
dfSusan.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
0,food,0.25,2019-02-19,41.85,ADM,False,food-ADM
1,food,0.25,2019-02-20,41.97,ADM,False,food-ADM
2,food,0.25,2019-02-21,42.57,ADM,False,food-ADM
3,food,0.25,2019-02-22,42.49,ADM,False,food-ADM
4,food,0.25,2019-02-25,42.43,ADM,False,food-ADM


In [15]:
dfPeter = df.loc[df["portfolio_name"] == "communication"]
dfPeter.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
2220,communication,0.25,2019-02-19,348.43,CHTR,False,communication-CHTR
2221,communication,0.25,2019-02-20,349.76,CHTR,False,communication-CHTR
2222,communication,0.25,2019-02-21,349.54,CHTR,False,communication-CHTR
2223,communication,0.25,2019-02-22,350.08,CHTR,False,communication-CHTR
2224,communication,0.25,2019-02-25,348.90,CHTR,False,communication-CHTR


In [16]:
dfTaylor = df.loc[df["portfolio_name"] == "financial"]
dfTaylor.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
4440,financial,0.25,2019-02-19,29.13,BAC,False,financial-BAC
4441,financial,0.25,2019-02-20,29.41,BAC,False,financial-BAC
4442,financial,0.25,2019-02-21,29.29,BAC,False,financial-BAC
4443,financial,0.25,2019-02-22,29.08,BAC,False,financial-BAC
4444,financial,0.25,2019-02-25,29.27,BAC,False,financial-BAC


In [17]:
dfCollin = df.loc[df["portfolio_name"] == "hedge"]
dfCollin.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
6660,hedge,0.25,2019-02-19,142.69,ETHUSD,False,hedge-ETHUSD
6661,hedge,0.25,2019-02-20,147.19,ETHUSD,False,hedge-ETHUSD
6662,hedge,0.25,2019-02-21,144.21,ETHUSD,False,hedge-ETHUSD
6663,hedge,0.25,2019-02-22,146.68,ETHUSD,False,hedge-ETHUSD
6664,hedge,0.25,2019-02-25,137.64,ETHUSD,False,hedge-ETHUSD


In [25]:
def get_beta_plot():
    beta_plot = dfJohn.hvplot.bar(
        x="ticker_name",
        y="close_price",
        title="Correlation Between Number of Violent Crimes and Murder",
        width=1000,
    ).opts(yformatter="%.0f")
    return beta_plot

get_beta_plot()

:Bars   [ticker_name]   (close_price)

In [ ]:
get_violent_murder():
    violent_murder = population_crime.hvplot.scatter

In [ ]:
geo_column = pn.Column(
    "## Beta", (), get_crime_plot()
)
​
scatter_column = pn.Column(
    "## Std Dev.",
    get_population_violence(),
    get_violent_murder(),
)
​
# Create tabs
crime_pop_dashboard = pn.Tabs(
    ("Geospatial", geo_column), ("Correlations", scatter_column)
)